# Описание данных


1. marketing_dealer - список дилеров;

2. marketing_dealerprice - результат работы парсера площадок дилеров:
      * product_key - уникальный номер позиции;
      * price - цена;
      * product_url - адрес страницы, откуда собраны данные;
      * product_name - заголовок продаваемого товара;
      * date - дата получения информации;
      * dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)

3. marketing_product - список товаров, которые производит и распространяет заказчик:
      * article - артикул товара;
      * ean_13 - код товара (см. EAN 13)
      * name - название товара;
      * cost - стоимость;
      * recommended_price - рекомендованная цена;
      * category_id - категория товара;
      * ozon_name - названиет товара на Озоне;
      * name_1c - название товара в 1C;
      * wb_name - название товара на Wildberries;
      * ozon_article - описание для Озон;
      * wb_article - артикул для Wildberries;
      * ym_article - артикул для Яндекс.Маркета;
       
4. marketing_productdealerkey - таблица матчинга товаров заказчика и товаров дилеров
      * key - внешний ключ к marketing_dealerprice
      * product_id - внешний ключ к marketing_product
      * dealer_id - внешний ключ к marketing_dealer
       

In [1]:
import pandas as pd
import os.path
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Функция для добавления пропущенных пробелов в наименованиях.
def add_spaces(text):
    spaced_text = re.sub(r'(?<=[a-zA-Z])(?=[а-яА-ЯёЁ])|(?<=[а-яА-ЯёЁ])(?=[a-zA-Z])', ' ', text)
    spaced_text = re.sub(r'(\S)\*(\S)', r'\1 * \2', spaced_text)
    spaced_text = re.sub(r'(\d+)([а-яА-ЯёЁa-zA-Z]+)', r'\1 \2', spaced_text)
    return spaced_text


# Функция для извлечения концентрации, объема, единицы измерения и фасовки из наименования товара в отдельные столбцы.
def extract_info(data):
    pattern_volume = r'\s*(\d+(?:[.,]\d+)?)\s*([мк]?[лг]|лит[р]?[ы]?|к[г]?[р]?[ам]?[ы]?)\s*'
    pattern_concentration = r'\s*(\d+:\d+)\s*'
    pattern_quantity = r'\s*(\d+)\s*(?:шт(?:ук[и]?|\.)?|ШТ|шт)\s*'

    data['volume'] = data['name'].str.extract(pattern_volume, expand=True)[0]
    data['units_of_meas'] = data['name'].str.extract(pattern_volume, expand=True)[1]
    data['concent'] = data['name'].str.extract(pattern_concentration, expand=False)
    data['units'] = data['name'].str.extract(pattern_quantity, expand=False)
    
    data['units'].fillna('1', inplace=True)
    data['concent'].fillna('1:1', inplace=True)
    data['volume'].fillna('1', inplace=True)
    data['units_of_meas'].fillna('шт', inplace=True)
    
    # Добавляем ваш фрагмент кода для удаления лишних пробелов между буквой и словом
    data['name'] = data['name'].str.replace(r'(\w)\s+(?=\w)', r'\1')

    return data


# Функция для извлечения первых 3-4 слов из названия товара
def extract_keywords(name):
    words = name.split()[:4]
    return ' '.join(words)


# Функция очистки,  токенизации, лемматизации и удаление стоп-слов.
def preprocess_text(text):
    # Очистка текста
    cleaned_text = re.sub(r"[^a-zA-Zа-яА-ЯёЁ ]", ' ', text)
    
    # Токенизация
    tokens = word_tokenize(cleaned_text.lower())
    
    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Удаление стоп-слов
    stop_words = set(stopwords.words('russian') + stopwords.words('english'))
    filtered_words = [lemma for lemma in lemmas if lemma not in stop_words]
    
    # Возвращение предобработанного текста
    return ' '.join(filtered_words)

In [3]:
data_dir = 'C:/Users/tra88/Downloads/Prosept'
try:
    df_a = os.path.join(data_dir, 'marketing_dealer.csv')
    dealers_id = pd.read_csv(df_a, sep=';')
    
    df_b = os.path.join(data_dir, 'marketing_dealerprice.csv')
    data_dealers = pd.read_csv(df_b, sep=';', parse_dates=['date'])
    
    df_c = os.path.join(data_dir, 'marketing_product.csv')
    data_products = pd.read_csv(df_c, sep=';', index_col='Unnamed: 0')
    
    df_d = os.path.join(data_dir, 'marketing_productdealerkey.csv')
    data_key = pd.read_csv(df_d, sep=';')
    
except:
    dealers_id = pd.read_csv('marketing_dealer.csv', sep=';')
    data_dealers = pd.read_csv('marketing_dealerprice.csv', sep=';')
    data_products = pd.read_csv('marketing_product.csv', sep=';', index_col='Unnamed: 0')
    data_key = pd.read_csv('marketing_productdealerkey.csv', sep=';')

In [4]:
dealers_id

,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
3,5,Castorama
4,6,Cubatora
5,7,Komus
6,9,Megastroy
7,10,OnlineTrade
8,11,Petrovich
9,12,sdvor


In [5]:
dealers_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18 non-null     int64 
 1   name    18 non-null     object
dtypes: int64(1), object(1)
memory usage: 420.0+ bytes


**Вывод о dealers_id:** 


В датафрейме dealers_id содержит 18 строк и 2 столбца. Пропуски и дубликаты в датафрейме отстутствуют.

In [6]:
data_dealers.sample(10)

,id,product_key,price,product_url,product_name,date,dealer_id
2334,2333,100121701,1354.0,https://www.bafus.ru/100121701/,Просепт Professional Multipower Bright средств...,2023-07-12,3
14457,14522,700000620,729.0,https://baucenter.ru/sredstva-ot-pleseni-gribk...,Средство для защиты от плесени PROSEPT FUNGI S...,2023-07-25,4
3623,3745,856995,1798.0,https://akson.ru//p/shpatlevka_dlya_plit_osb_p...,Шпаклевка для OSB-плит акриловая PROSEPT Propl...,2023-07-13,2
2692,2670,1656315,308.0,https://www.onlinetrade.ru/catalogue/sredstva_...,"Удалитель плесени PROSEPT FUNGI CLEAN, 500 мл,...",2023-07-12,10
10100,10242,100121836,436.0,https://www.bafus.ru/100121836/,Просепт Professional Crystal Black Active жидк...,2023-07-19,3
12910,13009,100121788,6581.0,https://www.bafus.ru/100121788/,Просепт Professional Duty Mud низкопенный конц...,2023-07-24,3
11569,11710,100121705,5107.0,https://www.bafus.ru/100121705/,Просепт Professional Polish 250 полимерное пок...,2023-07-21,3
17622,17775,344052,195.0,https://kazan.megastroy.com/products/344052,Средство д/мытья полов PROSEPT Multipower Floo...,2023-07-28,9
12462,12564,44231934,823.0,https://www.wildberries.ru/catalog/44231934/de...,Концентрат ULTRA,2023-07-21,1
13269,13335,1302289,469.0,https://www.onlinetrade.ru/catalogue/sredstva_...,Средство для удаления плесени PROSEPT Bath Fun...,2023-07-24,10


In [7]:
data_dealers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20416 entries, 0 to 20415
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            20416 non-null  int64         
 1   product_key   20416 non-null  object        
 2   price         20416 non-null  float64       
 3   product_url   20182 non-null  object        
 4   product_name  20416 non-null  object        
 5   date          20416 non-null  datetime64[ns]
 6   dealer_id     20416 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 1.1+ MB


In [8]:
df = data_dealers.query('product_url.isna()')
df['dealer_id'].unique()

array([7], dtype=int64)

Видно, что имеются пропуски. Посмотрим процент этих пропусков.

In [9]:
(data_dealers.isna().sum() / len(data_dealers)) * 100

id              0.00000
product_key     0.00000
price           0.00000
product_url     1.14616
product_name    0.00000
date            0.00000
dealer_id       0.00000
dtype: float64

In [10]:
print('Количество явных дубликатов: ', data_dealers.duplicated().sum())

Количество явных дубликатов:  0


Посмотрим количество уникальных наименований товаров

In [11]:
print('Количество уникальных наименований столбца product_name: ', len(data_dealers['product_name'].unique()))

Количество уникальных наименований столбца product_name:  1953


**Вывод о data_dealers:**

1. Датафрейм содержит 7 столбцов и 20416 строк.
2. Дубликаты не обнаружены.
3. Имеются пропуски в столбце product_url (1,2%)

Изучим информацию в датафрейме data_products.

In [12]:
data_products.sample(10)

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
114,191,D1 02500,4.610093e+12,"Дезинфицирующее средство ПРОФ ДЗ ""PROF DZ"" гот...",156.00,0.00,21.0,"Дезинфицирующее средство PROSEPT PROF DZ, 500 мл.","Дезинфицирующее средство PROSEPT PROF DZ, 500 мл.","Дезинфицирующее средство PROSEPT PROF DZ, 500 мл.",NaN,150119532.0,D1-02500,NaN
196,391,019-1,4.680008e+12,Удалитель плесени FUNGI CLEAN концентрат 1:1 /...,212.00,506.00,35.0,"Удалитель плесени PROSEPT FUNGI CLEAN, концент...","Удалитель плесени PROSEPT FUNGI CLEAN, концент...","Удалитель плесени PROSEPT FUNGI CLEAN, концент...",189522721.0,149974141.0,019-1,NaN
270,28,231-1,4.680008e+12,Концентрат эконом-класса для мытья половMultip...,88.79,208.00,50.0,Средство для мытья полов всех типов PROSEPT Mu...,Средство для мытья полов всех типов PROSEPT Mu...,Средство для мытья полов всех типов PROSEPT Mu...,451081249.0,151231991.0,231-1,NaN
495,52,113-075,4.680008e+12,Средство усиленного действия для удаления ржав...,75.00,176.00,52.0,Усиленное средство для удаления ржавчины и мин...,Усиленное средство для удаления ржавчины и мин...,Усиленное средство для удаления ржавчины и мин...,413264558.0,149811041.0,113-075,NaN
450,64,189-5,4.680008e+12,Средство для чистки акриловых поверхностейBath...,456.61,1067.00,46.0,Средство для чистки акриловых ванн и душевых к...,Средство для чистки акриловых ванн и душевых к...,Средство для чистки акриловых ванн и душевых к...,451565754.0,149811057.0,189-5,NaN
447,9,107-5,4.680008e+12,Универсальное моющее средство с дезинфицирующи...,594.00,1388.00,56.0,Моющее средство с дезинфицирующим эффектом PRO...,Моющее средство с дезинфицирующим эффектом PRO...,Моющее средство с дезинфицирующим эффектом PRO...,449936777.0,149811021.0,107-5,NaN
421,38,246-1,4.680008e+12,Концентрат для мытья половMULTIPOWER с аромато...,101.94,239.00,50.0,Средство для мытья полов PROSEPT Multipower ci...,Средство для мытья полов PROSEPT Multipower ci...,Средство для мытья полов PROSEPT Multipower ci...,417953521.0,151231986.0,246-1,NaN
259,290,025-5,4.680008e+12,Антисептик против грибка и плесени АНТИПЛЕСЕНЬ...,251.00,598.00,20.0,Антисептик универсальный против грибка и плесе...,Антисептик универсальный против грибка и плесе...,Антисептик универсальный против грибка и плесе...,189522744.0,150033472.0,025-5,NaN
222,467,М015-12,4.610093e+12,"Герметик акриловый цвет орех, ф/п 600мл (12 шт...",3686.40,7716.96,25.0,Герметик акриловый для швов для деревянных дом...,Герметик акриловый для швов для деревянных дом...,Герметик акриловый для швов для деревянных дом...,189522825.0,161240419.0,M015-12,NaN
339,380,046-3,4.680008e+12,"Грунт АКВАИЗОЛ, розовый, концентрат 1:9 / 3 л",658.00,1485.00,26.0,"Грунт влагоизолирующий PROSEPT Акваизол, 3 л.","Грунт влагоизолирующий PROSEPT Акваизол, 3 л.","Грунт влагоизолирующий PROSEPT Акваизол, 3 л.",453014587.0,149699629.0,046-3,NaN


In [13]:
data_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 496 entries, 0 to 495
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 496 non-null    int64  
 1   article            496 non-null    object 
 2   ean_13             464 non-null    float64
 3   name               494 non-null    object 
 4   cost               491 non-null    float64
 5   recommended_price  491 non-null    float64
 6   category_id        447 non-null    float64
 7   ozon_name          458 non-null    object 
 8   name_1c            485 non-null    object 
 9   wb_name            455 non-null    object 
 10  ozon_article       365 non-null    float64
 11  wb_article         340 non-null    float64
 12  ym_article         337 non-null    object 
 13  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(1), object(7)
memory usage: 58.1+ KB


Видно, что в столбцах имеются пропуски посмотрим, какое процент этих пропусков.

In [14]:
(data_products.isna().sum() / len(data_products)) *100

id                    0.000000
article               0.000000
ean_13                6.451613
name                  0.403226
cost                  1.008065
recommended_price     1.008065
category_id           9.879032
ozon_name             7.661290
name_1c               2.217742
wb_name               8.266129
ozon_article         26.411290
wb_article           31.451613
ym_article           32.056452
wb_article_td        93.548387
dtype: float64

Есть большое количество пропусков в столбце wb_article_td (почти 94%). Также достаточно большой обьем пропусков содержится в столбцах с артикулами (ozon_article, wb_article, ym_article). В этих столбцах обьем процент пропусков находится в диапазоне от 26,4% до 32,1%.   

 Столбец wb_article_td имеет смысл удалить, поскольку восстановить такой обьем пропусков практически невозможно. Поэтому лучше убрать столбцы ean_13, wb_article, ozon_article, ym_article, wb_article_td.

In [15]:
data_products = data_products.drop(['wb_article_td', 'ean_13', 'ozon_article', 'wb_article', 'ym_article'], axis=1)

In [16]:
data_products.query('recommended_price == 0.0')

,id,article,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name
45,187,D1 01050,"Кожный антисептик ПРОФ ДЗ ""PROF DZ"" готовый со...",44.0,0.0,21.0,"Кожный антисептик ПРОФ ДЗ ""PROF DZ"" готовый со...","Кожный антисептик ПРОФ ДЗ ""PROF DZ"" готовый со...","Кожный антисептик ПРОФ ДЗ ""PROF DZ"" готовый со..."
59,189,D1 01001,"Кожный антисептик ПРОФ ДЗ ""PROF DZ""готовый сос...",240.0,0.0,21.0,"Кожный антисептик PROSEPT PROF DZ, 1 л.","Кожный антисептик PROSEPT PROF DZ, 1 л.","Кожный антисептик PROSEPT PROF DZ, 1 л."
113,190,D1 01005,"Кожный антисептик ПРОФ ДЗ ""PROF DZ"" готовый со...",1164.0,0.0,21.0,"Кожный антисептик PROSEPT PROF DZ, 5 л.","Кожный антисептик PROSEPT PROF DZ, 5 л.","Кожный антисептик PROSEPT PROF DZ, 5 л."
114,191,D1 02500,"Дезинфицирующее средство ПРОФ ДЗ ""PROF DZ"" гот...",156.0,0.0,21.0,"Дезинфицирующее средство PROSEPT PROF DZ, 500 мл.","Дезинфицирующее средство PROSEPT PROF DZ, 500 мл.","Дезинфицирующее средство PROSEPT PROF DZ, 500 мл."
135,192,D1 02005,"Дезинфицирующее средство ПРОФ ДЗ ""PROF DZ"" гот...",1164.0,0.0,21.0,"Дезинфицирующее средство ПРОФ ДЗ ""PROF DZ"" гот...","Дезинфицирующее средство ПРОФ ДЗ ""PROF DZ"" гот...","Дезинфицирующее средство ПРОФ ДЗ ""PROF DZ"" гот..."
145,188,D1 01100,"Кожный антисептик ПРОФ ДЗ ""PROF DZ"" готовый со...",56.0,0.0,21.0,"Кожный антисептик ПРОФ ДЗ ""PROF DZ"" готовый со...","Кожный антисептик ПРОФ ДЗ ""PROF DZ"" готовый со...","Кожный антисептик ПРОФ ДЗ ""PROF DZ"" готовый со..."


In [17]:
data_products.query('cost.isna() or recommended_price.isna()')

,id,article,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name
4,502,0024-7 б,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN
23,503,0024-7 о,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,504,w022-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,449,0024-06 м12,"Герметик акриловый цвет Медовый 0,6 л (12 шт)",NaN,NaN,25.0,NaN,"Герметик акриловый цвет Медовый 0,6 л (12 шт)",NaN
110,454,0024-06 о12,"Герметик акриловый цвет Орех, ф/п 600мл (12 шт...",NaN,NaN,25.0,NaN,"Герметик акриловый цвет сосна, ф/п 600мл (12 ш...",NaN


In [18]:
data_products.dropna(subset=['name'], inplace=True)

In [19]:
data_products.query('name_1c.isna()')

,id,article,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name
4,502,0024-7 б,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN
31,118,273-20,Средство усиленного действия для чистки коптил...,3730.0,8712.0,NaN,NaN,NaN,NaN
32,119,274-20,Средство низкопенное для очистки алюминия и...,2818.0,6581.0,NaN,NaN,NaN,NaN
46,182,P1 11005,Cредство для рук гигиеническое с антибактериал...,565.0,1319.0,NaN,NaN,NaN,NaN
47,181,P1 11800,Cредство для рук гигиеническое с антибактериал...,155.0,361.0,NaN,NaN,NaN,NaN
60,186,P1 06005,Жидкое мыло с антибактериальным компонентом дл...,492.0,1067.0,NaN,NaN,NaN,NaN
98,436,Р1 09005,,500.0,600.0,NaN,NaN,NaN,NaN
102,433,D1 01050-3,"Кожный антисептик PROF DZ готовый состав / 0,1 л",112.8,226.0,NaN,NaN,NaN,NaN
115,456,М007-2,"Антисептик для рук PROSEPT PROF DZ, 2 штуки*10...",112.0,224.0,21.0,"Антисептик для рук PROSEPT PROF DZ, 2 штуки*10...",NaN,NaN


In [20]:
empty_name_rows = data_products[data_products['name'].str.contains(r'^\s*$')]
empty_name_rows

,id,article,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name
98,436,Р1 09005,,500.0,600.0,NaN,NaN,NaN,NaN


In [21]:
data_products.drop(index=empty_name_rows.index, inplace=True)

In [22]:
data_products['name'].unique()

array(['Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10  / 1 л',
       'Антигололед - 32 PROSEPTготовый состав / 12 кг',
       'Герметик акриловый цвет сосна, ф/п 600мл',
       'Кондиционер для белья с ароматом  королевского ИрисаCrystal Rinserконцентрат / 2 л ',
       'Герметик акриловой  цвет Белый, 7 кг',
       'Грунт БЕТОНКОНТАКТготовый состав / 6 кг',
       'Грунт БЕТОНКОНТАКТготовый состав / 12 кг',
       'Средство для удаления технических масел, смазочных материалов и нефтепродуктовDuty Oil концентрат 1:20-1:150 /  5 л',
       'Антисептик универсальный суперсильный',
       'Средство для мытья светлых полов с отбеливающим эффектомMultipower Whiteконцентрат 1:20-1:200 / 1 л',
       'Отбеливатель для древесины PROSEPT 50концентрат 1:1 / 5 л',
       'Антисептик лессирующий BiO LASUR / тик / 9 л',
       'Антисептик лессирующий BiO LASUR / махагон / 0,9 л',
       'Антисептик лессирующий BiO LASUR / лиственница / 0,9 л',
       'Антисептик лессирующий BiO LASUR / тик /

В наименованих иногда пропущены пробелы между слов. Необходимо добавить пропущенные пробелы в наименования товаров в столбцах 'name', 'name_1c', 'ozon_name' и 'wb_name'.

In [23]:
columns_to_apply = ['name', 'name_1c', 'ozon_name', 'wb_name']

data_products[columns_to_apply] = data_products[columns_to_apply].astype(str).applymap(add_spaces)

Поскольку в названиях товаров присутствуют обьем, количество штук в упаковке и концентрация продукция. Также у каждого обьема разная единица измерения. Эти данные можно извлечь и зафиксировать в отдельные столбцы в качестве дополнительных признаков.

In [24]:
data_products = extract_info(data_products)

При анализе уникальных значений в столбце name обнаружено, что есть лишний пробел в слове Средство. Посмотрим, много ли таких позиций и произведем удаление этого пробела в слове.

In [25]:
# Создание фильтра, который проверяет, содержит ли каждая строка в столбце 'name' подстроку "C редство"
filter = data_products['name'].str.contains('C редство')

# Применение фильтра к датафрейму для вывода только строк, которые соответствуют фильтру
data_products[filter]

,id,article,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,volume,units_of_meas,concent,units
28,78,143-20,C редство для чистки коптильных камер Cooky Sm...,2781.00,6496.0,49.0,C редство для чистки коптильных камер Cooky Sm...,C редство для чистки коптильных камер Cooky Sm...,C редство для чистки коптильных камер Cooky Sm...,20,л,1:10,1
46,182,P1 11005,C редство для рук гигиеническое с антибактериа...,565.00,1319.0,NaN,nan,nan,nan,5,л,1:1,1
47,181,P1 11800,C редство для рук гигиеническое с антибактериа...,155.00,361.0,NaN,nan,nan,nan,"0,8",л,1:1,1
452,77,143-5,C редство для чистки коптильных камер Cooky Sm...,726.52,1696.0,40.0,Профессиональное средство для чистки коптильны...,Профессиональное средство для чистки коптильны...,Профессиональное средство для чистки коптильны...,5,л,1:10,1


In [26]:
data_products['ozon_name'] = data_products['ozon_name'].str.replace('C редство', 'Средство')
data_products['name_1c'] = data_products['name_1c'].str.replace('C редство', 'Средство')
data_products['wb_name'] = data_products['wb_name'].str.replace('C редство', 'Средство')
data_products['name'] = data_products['name'].str.replace('C редство', 'Средство')

Так же в наименования оказались еще места, которые не были обработаны функцией и уберем эти проблемы вручную.

In [27]:
data_products['name'] = data_products['name'].str.replace('БЕТОНКОНТАКТготовый', 'БЕТОНКОНТАКТ готовый')
data_products['name'] = data_products['name'].str.replace('"к', '" к')
data_products['name'] = data_products['name'].str.replace('яблокаконцентрированное', 'яблока концентрированное')
data_products['name'] = data_products['name'].str.replace('(сухой остаток 20%)', ' (сухой остаток 20%) ')
data_products['name'] = data_products['name'].str.replace('.C', '. C')

Проверим результаты обработки.

In [28]:
data_products['name'].unique()

array(['Антисептик невымываемый PROSEPT ULTRA концентрат 1:10  / 1 л',
       'Антигололед - 32 PROSEPT готовый состав / 12 кг',
       'Герметик акриловый цвет сосна, ф/п 600 мл',
       'Кондиционер для белья с ароматом  королевского Ириса Crystal Rinser концентрат / 2 л ',
       'Герметик акриловой  цвет Белый, 7 кг',
       'Грунт БЕТОНКОНТАКТ готовый состав / 6 кг',
       'Грунт БЕТОНКОНТАКТ готовый состав / 12 кг',
       'Средство для удаления технических масел, смазочных материалов и нефтепродуктов Duty Oil концентрат 1:20-1:150 /  5 л',
       'Антисептик универсальный суперсильный',
       'Средство для мытья светлых полов с отбеливающим эффектом Multipower White концентрат 1:20-1:200 / 1 л',
       'Отбеливатель для древесины PROSEPT 50 концентрат 1:1 / 5 л',
       'Антисептик лессирующий BiO LASUR / тик / 9 л',
       'Антисептик лессирующий BiO LASUR / махагон / 0,9 л',
       'Антисептик лессирующий BiO LASUR / лиственница / 0,9 л',
       'Антисептик лессирующий BiO L

Проблемы устранены.

Пропущеные категории можно восполнить по схожим товарам из этой же категории.

In [29]:
# Заполнение пропущенных значений в категориях на основе группировки по первым 3-4 словам в названиях товаров
data_products['category_id'].fillna(data_products['name'].apply(extract_keywords), inplace=True)

Для пропусков в ценах поставим просто заглушку -1.

In [30]:
data_products['cost'].fillna('-1', inplace=True)
data_products['recommended_price'].fillna('-1', inplace=True)

In [31]:
(data_products.isna().sum() / len(data_products)) *100

id                   0.0
article              0.0
name                 0.0
cost                 0.0
recommended_price    0.0
category_id          0.0
ozon_name            0.0
name_1c              0.0
wb_name              0.0
volume               0.0
units_of_meas        0.0
concent              0.0
units                0.0
dtype: float64

In [32]:
data_key

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403
...,...,...,...,...
1695,2019,530017190,18,267
1696,2020,530139882,18,286
1697,2021,534659036,18,129
1698,2022,898350801,18,1


In [33]:
data_key.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1700 non-null   int64 
 1   key         1700 non-null   object
 2   dealer_id   1700 non-null   int64 
 3   product_id  1700 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 53.3+ KB


In [34]:
# Список наименований товаров из таблицы data_products
product_names = data_products['name'].tolist()

# Предобработка наименований товаров
preprocessed_names = [preprocess_text(name) for name in product_names]

# Преобразование векторизации
vectorizer = TfidfVectorizer()
vectorized_names = vectorizer.fit_transform(preprocessed_names).toarray()

# Применение векторизации к таблице data_products
data_products['vectorized_name'] = list(vectorized_names)

# Применение векторизации к таблице data_dealers
data_dealers['vectorized_name'] = list(vectorizer.transform(data_dealers['product_name'].apply(preprocess_text)).toarray())

In [35]:
# Поиск топ 5 наиболее подходящих товаров для каждого товара из таблицы data_dealers
top_matches = []
for index, row in data_dealers.iterrows():
    query_vector = row['vectorized_name']
    similarity_scores = data_products['vectorized_name'].apply(lambda x: (x * query_vector).sum())
    top_matches_indices = similarity_scores.nlargest(5).index.tolist()
    top_matches.append(data_products.loc[top_matches_indices, 'name'].tolist())

In [36]:
# Создание нового датафрейма с результатами
results = pd.DataFrame({'product_name': data_dealers['product_name'], 'top_matches': top_matches})

# Преобразование столбца top_matches в отдельные столбцы
results_expanded = results['top_matches'].apply(pd.Series)

# Объединение результатов с исходным датафреймом
results = pd.concat([results['product_name'], results_expanded], axis=1)

In [37]:
results.sample(10)

,product_name,0,1,2,3,4
2324,Просепт Professional Duty Graffiti средство дл...,"Средство для удаления граффити, маркера, краск...","Средство для удаления граффити, маркера, краск...","Средство для удаления граффити, маркера, краск...","Набор DUTY GRAFFITI, 0.4 л, 2 шт",Средство для удаления наклеек Duty Universal /...
14385,Просепт Professional Duty White удалитель стро...,Средство для удаления гипсовой пыли Duty White...,Средство для удаления гипсовой пыли Duty White...,Удалитель мха концентрат 1:1 / 1 л,"Набор Удалитель мха, концентрат 1 л, 2 шт",Удалитель мха готовый состав / 5 л
12845,Просепт Multipower Wood моющее средство для ба...,Моющее средство для бани и сауны Multipower Wo...,"Набор для бани (Universal Wood, Multipower Wood)",Лак для бани и сауны PROSEPT готовый состав / ...,Лак для бани и сауны PROSEPT готовый состав / ...,Антисептик для бани и сауны ECO SAUNA готовый ...
15540,Просепт Super Rubber краска резиновая (12 кг) ...,Краска резиновая зеленый мох Ral 6005 / 12 кг,Краска резиновая зеленый мох Ral 6005 / 3 кг,Краска резиновая зеленый мох Ral 6005 / 1 кг,Краска резиновая белый Ral 9003 / 12 кг,Краска резиновая белый Ral 9003 / 3 кг
4538,Жидкое мыло Diona Antibac с антибактериальным ...,Антибактериальное мыло Diona Antibac готовое с...,Антибактериальное мыло Diona Antibac готовое с...,Антибактериальное мыло Diona Antibac готовое с...,Жидкое мыло с антибактериальным компонентом дл...,"Жидкое гель-мыло с перламутром. Без цвета, без..."
6621,Кондиционер для белья PROSEPT Crystal Rinser К...,Кондиционер для белья без красителей и аромати...,Кондиционер для белья без красителей и аромати...,"Кондиционер для белья с ароматом ""Альпийская с...","Кондиционер для белья с ароматом ""Альпийская с...",Кондиционер для белья с ароматом экзотических ...
8756,Просепт Professional Bath Acid Plus усиленное ...,Усиленное средство для уборки после строительс...,Средство усиленного действия для удаления ржав...,Средство усиленного действия для удаления ржав...,Средство усиленного действия для удаления ржав...,Средство для ухода за мебелью (полироль)Univer...
15446,Просепт Bio Lasur антисептик лессирующий защит...,Антисептик лессирующий BiO LASUR / бесцветный ...,Антисептик лессирующий BiO LASUR / бесцветный ...,Антисептик лессирующий BiO LASUR / бесцветный ...,Антисептик лессирующий BiO LASUR / орех / 9 л,"Антисептик лессирующий BiO LASUR / орех / 0,9 л"
3797,Просепт 50 отбеливатель для древесины (1 л) же...,Отбеливатель для древесины PROSEPT 50 концентр...,Отбеливатель для древесины PROSEPT 50 концентр...,Отбеливатель для древесины PROSEPT 50 концентр...,Отбеливатель для древесины PROSEPT 50 концентр...,Отбеливатель для древесины PROSEPT 50 концентр...
16133,"Чистящее средство PROSEPT Bath Uni, с антимикр...",Универсальный концентрат для санитарных комнат...,Универсальный концентрат для санитарных комнат...,Средство для удаления плесени с дезинфицирующ...,Средство для удаления плесени с дезинфицирующ...,Усиленное чистящее средство с антистатическим ...
